# IMPORTING PACKAGES AND DATASET

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

drive.mount("/content/drive", force_remount=True)
import pandas as pd

csv_path = "/content/drive/MyDrive/covid_data.csv"
data_frame = pd.read_csv(csv_path, encoding='ISO-8859-1')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


In [2]:
data_frame.columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Date', 'Confirmed',
       'Deaths'],
      dtype='object')

#Converting String values to Integer

In [3]:
from sklearn import preprocessing
string_to_int= preprocessing.LabelEncoder()                
data_frame=data_frame.apply(string_to_int.fit_transform)

# **LISTING OUT THE FEATURES AND TARGET ATTRIBUTES**

In [4]:
Features = ['UID', 'Province_State', 'Country_Region', 'Confirmed']
Target = ["Deaths"]

# **SPLITTING THE DATASET INTO TRAINING AND TESTING PARTS**

In [5]:
#Split dataset to Training Set & Test Set
from sklearn.model_selection import train_test_split

X = data_frame[Features]
y = data_frame[Target]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                        train_size = 0.8, 
                        test_size = 0.2, 
                        random_state= 10)



x1 = X_train[Features]    #Features to train
x2 = y_train[Target]      #Target Class to test
y1 = X_test[Features]    #Features to test
y2 = y_test[Target]      #Target Class to test

# **CHI2 METHOD FOR SELECTION OF BEST FEATURES**

In [6]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
best_features= SelectKBest(score_func=chi2, k=3)
fit= best_features.fit(X,y)

In [7]:
df_scores= pd.DataFrame(fit.scores_*100)
df_columns= pd.DataFrame(X.columns)

In [8]:
features_scores= pd.concat([df_columns, df_scores], axis=1)
features_scores.columns= ['Features', 'Score']
features_scores.sort_values(by = 'Score')

,Features,Score
1,Province_State,5.960863e+06
0,UID,2.955877e+08
3,Confirmed,2.209765e+11
2,Country_Region,NaN


# **ANN CLASSIFICATION**

In [9]:
from keras.models import Sequential
from keras.layers import Dense
# create the ANN model
model = Sequential()
model.add(Dense(8, input_dim=4, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam')

# train the model
model.fit(X_train, y_train, epochs=40, batch_size=100)

# evaluate the model on the testing set
accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/40
5024/5024 [==============================] - 11s 2ms/step - loss: -3215862.7500
Epoch 2/40
5024/5024 [==============================] - 9s 2ms/step - loss: -18390764.0000
Epoch 3/40
5024/5024 [==============================] - 10s 2ms/step - loss: -44506312.0000
Epoch 4/40
5024/5024 [==============================] - 10s 2ms/step - loss: -80549984.0000
Epoch 5/40
5024/5024 [==============================] - 9s 2ms/step - loss: -127060600.0000
Epoch 6/40
5024/5024 [==============================] - 10s 2ms/step - loss: -183150720.0000
Epoch 7/40
5024/5024 [==============================] - 10s 2ms/step - loss: -248615152.0000
Epoch 8/40
5024/5024 [==============================] - 9s 2ms/step - loss: -324829440.0000
Epoch 9/40
5024/5024 [==============================] - 12s 2ms/step - loss: -411566272.0000
Epoch 10/40
5024/5024 [==============================] - 10s 2ms/step - loss: -508431840.0000
Epoch 11/40
5024/5024 [==============================] - 9s 2ms/step - loss: 

#KNN Classification

In [10]:
# K-Nearest Neighbors
# Create Model with configuration 
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=3)

# Model Training
knn_model.fit(X=x1,
             y=x2)

# Prediction
result = knn_model.predict(y1) 

/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
ac_sc = accuracy_score(y2, result)
rc_sc = recall_score(y2, result, average="weighted")
pr_sc = precision_score(y2, result, average="weighted")
f1_sc = f1_score(y2, result, average='micro')
confusion_m = confusion_matrix(y2, result)
print("========== KNN Results ==========")
print("Accuracy    : ", ac_sc)
print("Recall      : ", rc_sc)
print("Precision   : ", pr_sc)
print("F1 Score    : ", f1_sc)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


========== KNN Results ==========
Accuracy    :  0.8249378901770926
Recall      :  0.8249378901770926
Precision   :  0.8172503758572286
F1 Score    :  0.8249378901770926


#Ensemble Voting Model

In [15]:
# Ensemble Voting Model
# Combine 3 Models to create an Ensemble Model
# Create Model with configuration
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
eclf1 = VotingClassifier(estimators=[('knn', knn_model)], 
                         weights=[1],
                         flatten_transform=True)
eclf1 = eclf1.fit(X=x1, y=x2)   

# Prediction
result = eclf1.predict(y1)

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [16]:
# Model Evaluation
ac_sc = accuracy_score(y2, result)
rc_sc = recall_score(y2, result, average="weighted")
pr_sc = precision_score(y2, result, average="weighted")
f1_sc = f1_score(y2, result, average='micro')
confusion_m = confusion_matrix(y2, result)

print("========== Voting Model Results ==========")
print("Accuracy    : ", ac_sc)
print("Recall      : ", rc_sc)
print("Precision   : ", pr_sc)
print("F1 Score    : ", f1_sc)

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


========== Voting Model Results ==========
Accuracy    :  0.8249378901770926
Recall      :  0.8249378901770926
Precision   :  0.8172503758572286
F1 Score    :  0.8249378901770926
